<a href="https://colab.research.google.com/github/corricelli/NGSS-Dynamic-Data-Gem/blob/main/ddg_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GENERATE SYNTHETIC DATA ALIGNED WITH NGSS TOPICS**

This Colab handles two steps:
1) generate precise theoretical values based on scientific laws
2) introduce controlled, realistic variability

In [45]:
#CODE CELL 1: Setup and imports

import numpy as np
import pandas as pd
import gspread #library for interacting with Google Sheets
from scipy.optimize import curve_fit #useful to for advanced model fitting (HS students)
from google.colab import auth

print("Libraries loaded. Ready for scientific modeling.")

Libraries loaded. Ready for scientific modeling.


In [46]:
# CODE CELL 6 (CORRECTED): Google Authentication

# 1. Import module for credential defaults
from google.auth import default

# 2. Authenticate Colab to access Google services (this will prompt a sign-in in the browser)
# This uses the credentials of the logged-in Google account (your Google School account).
from google.colab import auth
auth.authenticate_user()

# 3. Retrieve default credentials (token)
creds, _ = default()

# 4. Use the authenticated credentials to create the gspread client instance.
# 'gc' is the authorized client object we will use in CODE CELL 7 to open the sheet.
gc = gspread.authorize(creds)

print("Google Authentication complete. Ready to access sheets.")

Google Authentication complete. Ready to access sheets.


In [47]:
# CODE CELL 2: Scientific Models (DCI Functions)
# logistic growth is related to population dynamics required in MS/HS-LS2-1

def logistic_growth_model(t, L, k, t0):
    """
    Calculates population count based on the Logistic Growth (S-curve) function.
    t: time array
    L: Carrying Capacity (max population)
    k: Growth Rate parameter
    t0: Time shift parameter (midpoint)
    """
    # Mathematical expression for the S-curve: D(t) = L / (1 + e^(-k * (t - t0))) [3]
    return L / (1.0 + np.exp(-k * (t - t0)))


def kinetic_energy_model(mass, velocity):
    """
    Calculates Kinetic Energy (KE) based on Newtonian mechanics (MS-PS3-1). [5]
    KE = 0.5 * m * v^2
    """
    # Calculate Kinetic Energy: KE is proportional to mass and the square of velocity
    return 0.5 * mass * (velocity**2)

In [48]:
# CODE CELL 7: Sheet Reading Logic

# --------------------------------------------------------------------------
# CRITICAL: PASTE YOUR UNIQUE SPREADSHEET ID HERE
# This ID is taken from the URL you provided: 1othRGkeNPnWQUpepGsNaM1F_OS-MWlDUdRND9-s7ul8
SPREADSHEET_ID = '1othRGkeNPnWQUpepGsNaM1F_OS-MWlDUdRND9-s7ul8'
# --------------------------------------------------------------------------


def get_latest_params_from_sheet():
    """
    Connects to the specified Google Sheet via its ID and reads the latest
    parameters from the first worksheet (Sheet1).

    Returns: A dictionary of clean parameters, or None if reading fails.
    """
    # NOTE: 'gc' is the authorized client object defined in CODE CELL 6
    global gc, SPREADSHEET_ID

    try:
        # 1. Open the sheet by its unique ID
        worksheet = gc.open_by_key(SPREADSHEET_ID).sheet1

        # 2. Get all records as a list of dictionaries (using header names)
        list_of_records = worksheet.get_all_records()

        if not list_of_records:
            print("Error: Control Sheet is empty besides the header row.")
            return None

        # 3. The latest entry is the last dictionary in the list
        latest_params = list_of_records[-1]

        # 4. Standardize and type-convert the parameters (CRITICAL STEP)
        return {
            'PE_ID': latest_params.get('PE_ID'),
            # Convert string inputs from the sheet to floats for calculation.
            'L_param': float(latest_params.get('Param_L', 0)),
            'Noise_Sigma': float(latest_params.get('Noise_Sigma', 0)),

            # Use fixed defaults for variables not currently present in the sheet/form:
            'k_param': 0.7,
            't_range': 60,
            'Mass_Const': 10.0
        }

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"Error: Spreadsheet with ID '{SPREADSHEET_ID}' not found. Check the ID and sharing permissions.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during sheet reading: {e}")
        # This occurs if, for example, the PE_ID column header name is wrong.
        return None

In [49]:
# CODE CELL 3: DDG Core Engine with Noise Injection

def generate_ddg_data(pe_id, params):
    """
    Generates the final synthetic dataset based on NGSS PE ID and parameters.
    Applies the scientific model and injects controlled Gaussian noise.
    """
    # Extract core parameters from the input dictionary (from CODE CELL 7)
    pe_id_str = params.get('PE_ID')
    L_param = params.get('L_param', 5000.0)
    k_param = params.get('k_param', 0.7)
    t_range = params.get('t_range', 60)
    mass_const = params.get('Mass_Const', 10.0)
    sigma_noise = params.get('Noise_Sigma', 200.0) # Pedagogical Noise Factor [1]

    # 1. Define the Independent Variable Array (Time steps or Velocity points)
    t_array = np.arange(0, int(t_range), 1)

    # 2. Calculate Theoretical Data (The Scientific Signal)
    if pe_id_str == "LS2-1": # MS/HS Population Dynamics [2]
        # Use a sensible midpoint estimate for t0 for a clean S-curve [3]
        t0_estimate = t_range / 3.0
        D_theoretical = logistic_growth_model(t_array, L_param, k_param, t0_estimate)
        col_name = "Population_Count"
        meta_param_name = "Theoretical_Capacity (L)"
        meta_param_value = L_param

    elif pe_id_str == "PS3-1_KE": # MS/HS Kinetic Energy [4, 5]
        # Treat t_array as velocity (v)
        D_theoretical = kinetic_energy_model(mass_const, t_array)
        col_name = "Kinetic_Energy (J)"
        meta_param_name = "Mass_Const (kg)"
        meta_param_value = mass_const

    else:
        # Error handling for unrecognized PE_ID (uses confirmed working syntax)
        print(f"ERROR: Unrecognized PE ID: {pe_id_str}. Returning empty data.")
        return pd.DataFrame({"Error":''}) # [6]

    # 3. Apply Controlled Noise (Gaussian Distribution)
    # The standard normal distribution is centered at 0 and scaled by sigma_noise
    noise = np.random.normal(loc=0.0, scale=sigma_noise, size=len(t_array))

    # Calculate noisy data and ensure physical quantities are non-negative integers
    D_synthetic = np.clip((D_theoretical + noise), a_min=0, a_max=None).astype(int)

    # 4. Create Final DataFrame with Metadata
    final_data = pd.DataFrame({
        'Time_Step / Independent_Variable': t_array,
        col_name: D_synthetic,
        'PE_ID': pe_id_str,
        'Noise_Applied (Sigma)': sigma_noise,
        meta_param_name: meta_param_value
    })

    return final_data

In [50]:
# CODE CELL 4 (REVISED): Live Sheet Reading and Data Generation Test

# 1. Read the latest input data from the Google Sheet
# This calls the function defined in CODE CELL 7, which uses your SHEET_ID and gc client.
live_params = get_latest_params_from_sheet()

# 2. Check if the read was successful and proceed
if live_params:
    # Extract the PE ID (e.g., 'LS2-1' or 'PS3-1_KE')
    pe_to_run = live_params.get('PE_ID')

    # Print a summary of the parameters successfully read
    print("--- LIVE RUN START ---")
    print(f"Target Phenomenon: {pe_to_run}")
    print(f"DCI Parameter (L/Mass): {live_params['L_param']}")
    print(f"SEP Parameter (Noise): {live_params}")

    # 3. Generate the data using the live parameters.
    # The generate_ddg_data function is defined in CODE CELL 3.
    synthetic_df = generate_ddg_data(pe_to_run, live_params)

    # 4. Display the results
    print("\n--- SYNTHETIC DATA GENERATED ---")
    print(synthetic_df.head(15))
    print(f"\nData Shape: {synthetic_df.shape} (Number of rows generated)")

else:
    print("Could not retrieve parameters from Google Sheet. Aborting generation test.")

# The generated DataFrame 'synthetic_df' is now available for the next step (CODE CELL 5).

--- LIVE RUN START ---
Target Phenomenon: PS3-1_KE
DCI Parameter (L/Mass): 25.0
SEP Parameter (Noise): {'PE_ID': 'PS3-1_KE', 'L_param': 25.0, 'Noise_Sigma': 100.0, 'k_param': 0.7, 't_range': 60, 'Mass_Const': 10.0}

--- SYNTHETIC DATA GENERATED ---
    Time_Step / Independent_Variable  Kinetic_Energy (J)     PE_ID  \
0                                  0                 180  PS3-1_KE   
1                                  1                   0  PS3-1_KE   
2                                  2                 136  PS3-1_KE   
3                                  3                   0  PS3-1_KE   
4                                  4                  31  PS3-1_KE   
5                                  5                 320  PS3-1_KE   
6                                  6                 294  PS3-1_KE   
7                                  7                 165  PS3-1_KE   
8                                  8                 633  PS3-1_KE   
9                                  9               

In [51]:
# CODE CELL 5: Prepare Final Output (CSV String)

def prepare_csv_output(dataframe):
    """
    Converts the Pandas DataFrame to a CSV formatted string, ready for Apps Script transfer.
    """
    # Convert the DataFrame to a CSV string.
    # index=False ensures we do not include the Python row numbers in the output data.
    csv_string = dataframe.to_csv(index=False)

    return csv_string

# Run the preparation step using the DataFrame generated in CODE CELL 4
csv_data_string = prepare_csv_output(synthetic_df)

# Uncomment the line below if you want to visually inspect the raw CSV output string:
# print(csv_data_string)

print("\nData prepared as CSV string for Apps Script delivery.")


Data prepared as CSV string for Apps Script delivery.


In [52]:
# CODE CELL 8 (NEW): Final Data Write to Output Sheet

# --------------------------------------------------------------------------
# CRITICAL: PASTE YOUR UNIQUE OUTPUT SHEET ID HERE
# This ID is taken from the URL you provided: 1nWEXvgeKDfa1gQG0iFBK0mMiraWvu73QtJjMtGYD9oI
OUTPUT_SHEET_ID = '1nWEXvgeKDfa1gQG0iFBK0mMiraWvu73QtJjMtGYD9oI'
# --------------------------------------------------------------------------


def import_csv_to_sheet(sheet_id, csv_string):
    """
    Writes the generated CSV data string directly to the specified Google Sheet.
    This overwrites the content of the first worksheet ('Sheet1').
    """
    global gc
    try:
        # Open the target Output Sheet by its unique ID
        spreadsheet = gc.open_by_key(sheet_id)
        worksheet = spreadsheet.sheet1 # Assuming the first tab is where data goes

        # 1. Clear existing content to ensure a clean slate
        worksheet.clear()

        # 2. Convert the CSV string into a list of lists (required by gspread's batch update)
        import csv
        from io import StringIO

        data_io = StringIO(csv_string)
        reader = csv.reader(data_io)
        data_list = list(reader)

        # 3. Write the new data list (including headers) back to the sheet
        # This uses the A1 notation to start writing from the top-left corner
        worksheet.update('A1', data_list)

        print(f"\nSUCCESS: Data written to Output Sheet (ID: {sheet_id})")
        print(f"Data contains {len(data_list) - 1} rows.")
        return True

    except Exception as e:
        print(f"\nERROR during data write: {e}")
        return False

# Run the final write operation
import_csv_to_sheet(OUTPUT_SHEET_ID, csv_data_string)

/tmp/ipython-input-90103579.py:34: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_list)



SUCCESS: Data written to Output Sheet (ID: 1nWEXvgeKDfa1gQG0iFBK0mMiraWvu73QtJjMtGYD9oI)
Data contains 60 rows.


True